# Model 3: Using XGB Regressor

# Importing all the necessary libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import csv
import numpy as np
import operator
import random
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn.discriminant_analysis
import sklearn.linear_model as skl_lm


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import hamming_loss, accuracy_score 
from pandas import DataFrame
from datetime import datetime
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error


In [ ]:
PATH ='/kaggle/input/covid19-global-forecasting-week-4'
datatrain = pd.read_csv(f'{PATH}/train.csv')
datatest = pd.read_csv(f'{PATH}/test.csv')

In [ ]:
date = pd.to_datetime(datatrain["Date"])
datet = pd.to_datetime(datatest["Date"])
print (date)


In [ ]:
ldate = int(len(date))
ldatet = int(len(datet))
print("Length of training- date is", ldate)
print("Length of test- date is", ldatet)

In [ ]:
m = []
d = []
for i in range(0,ldate):
    dx = (date[i].strftime("%d"))
    mx = (date[i].strftime("%m"))
    m.append(int(mx))
    d.append(int(dx))

mt = []
dt = []
for i in range(0,ldatet):
    dtx = (datet[i].strftime("%d"))
    mtx = (datet[i].strftime("%m"))
    mt.append(int(mtx))
    dt.append(int(dtx))


In [ ]:
train = datatrain
test = datatest


In [ ]:
train.insert(6,"Month",m,False)
train.insert(7,"Day",d,False)
test.insert(4,"Month",mt,False)
test.insert(5,"Day",dt,False)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print("Datatrain")
traindays = datatrain['Date'].nunique()
print("Number of Country_Region: ", datatrain['Country_Region'].nunique())
print("Number of Province_State: ", datatrain['Province_State'].nunique())
print("Number of Days: ", traindays)

notrain = datatrain['Id'].nunique()
print("Number of datapoints in train:", notrain)
lotrain = int(notrain/traindays)
print("L Trains:", lotrain)


In [ ]:
print("Datatest")
testdays = datatest['Date'].nunique()
print("Number of Days: ", testdays)
notest = datatest['ForecastId'].nunique()
print("Number of datapoints in test:", notest)
lotest = int(notest/testdays)
print("L Test:", lotest)


In [ ]:

zt = datet[0]
daycount = []
for i in range(0,lotrain):
    for j in range(1,traindays+1):
        daycount.append(j)



In [ ]:
for i in range(traindays):
    if(zt == date[i]):
        zx = i
        print(zx)
        
daytest = []
for i in range(0,lotest):
    for j in range(1,testdays+1):
        jr = zx + j
        daytest.append(jr)


In [ ]:
train.insert(8,"DayCount",daycount,False)
test.insert(6,"DayCount",daytest,False)

In [ ]:
traincount = int(len(train["Date"]))

testcount = int(len(test["Date"]))



In [ ]:
train.Province_State = train.Province_State.fillna(0)
empty = 0
for i in range(0,traincount):
    if(train.Province_State[i] == empty):
        train.Province_State[i] = train.Country_Region[i]

In [ ]:
test.Province_State = test.Province_State.fillna(0)
empty = 0
for i in range(0,testcount):
    if(test.Province_State[i] == empty):
        test.Province_State[i] = test.Country_Region[i]

In [ ]:
label = preprocessing.LabelEncoder()
train.Country_Region = label.fit_transform(train.Country_Region)
train.Province_State = label.fit_transform(train.Province_State)

In [ ]:
test.Country_Region = label.fit_transform(test.Country_Region)
test.Province_State = label.fit_transform(test.Province_State)


In [ ]:
X = np.c_[train["Province_State"], train["Country_Region"], train["DayCount"], train["Month"], train["Day"]]
Xt = np.c_[test["Province_State"], test["Country_Region"], test["DayCount"], test["Month"], test["Day"]]

In [ ]:
display(X.shape)
display(Xt.shape)

In [ ]:
Y1 = train["ConfirmedCases"]
Y2 = train["Fatalities"]

In [ ]:
display(Y1.shape)
display(Y2.shape)

In [ ]:
Y1.head(100)
#Y1.shape
Y2.head(100)

In [ ]:
def rmsle(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred)**(1/2);

# Building the XGB Regressor Model

* # Default settings:

(n_estimators = 1000, 

gamma = 0, 

learning_rate = 0.05,  

random_state = 42 , 

max_depth = 20)


In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 1000, gamma = 0, learning_rate = 0.05,  
                                  random_state = 42 , max_depth = 20) 
Regressor_fatalities = XGBRegressor(n_estimators = 1000 , gamma = 0, learning_rate = 0.05,  
                                    random_state = 42 , max_depth = 20)

In [ ]:
import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


In [ ]:
ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

In [ ]:
ypred.shape

In [ ]:
Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

In [ ]:
"""Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
yptest = Regressor_fatalities.predict(X)
yptest = np.round(yptest)
B = mean_squared_error(yptest,Y2)
print("Training - (Fatalities) Mean Squared Error is", B)"""


Training - (Fatalities) Mean Squared Error is 0.0

In [ ]:
ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

In [ ]:
final_pred = pd.DataFrame()
forecast = test["ForecastId"]

In [ ]:

#yp = yp.drop(['Country_Region', 'Date', 'Month', 'Day', 'DayCount', 'Province_State'], axis=1)


In [ ]:
final_pred.insert(0,"ForecastId",forecast,False)
final_pred.insert(1,"ConfirmedCases",ypred,False)
final_pred.insert(2, "Fatalities",ypred2,False)

In [ ]:
final_pred

In [ ]:
print(final_pred)

# Changing the Learning Rate:







learning_rate = 0.5

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 1000, gamma = 0, learning_rate = 0.5,  
                                  random_state = 42 ,max_depth = 20) 
Regressor_fatalities = XGBRegressor(n_estimators = 1000 , gamma = 0, learning_rate = 0.5,  
                                    random_state = 42 , max_depth = 20)


In [ ]:

import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 



# Changing the learning rate to 0.01

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 1000, gamma = 0, learning_rate = 0.01,  
                                  random_state = 42 , max_depth = 20) 
Regressor_fatalities = XGBRegressor(n_estimators = 1000 , gamma = 0, learning_rate = 0.01,  
                                    random_state = 42 , max_depth = 20)


import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# changing the learning rate to 0.99

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 1000, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 , max_depth = 20) 
Regressor_fatalities = XGBRegressor(n_estimators = 1000 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 , max_depth = 20)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# # Changing the n_estimators
n_estimators: number of trees you want to build.

n_estimators = 10

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 10, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 , max_depth = 20) 
Regressor_fatalities = XGBRegressor(n_estimators = 10 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 , max_depth = 20)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

In [ ]:
# Changing the n_estimators to 2500

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 , max_depth = 20) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 , max_depth = 20)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# The training time increase when we increase the number of n_estimators since it is the number of trees we want to build. However we need an ideal sweet spot such that we get low training error rate and acceptable training time

# Also there is a interdependence between the number of n_estimators i.e. number of trees used and learning rate. If we reduce the learning rate too much we need to add the number of trees.

# Changing the n_estimators to 5000

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 5000, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 , max_depth = 20) 
Regressor_fatalities = XGBRegressor(n_estimators = 5000 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 , max_depth = 20)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# The above scenario takes too much training time and the error is similar to the previous case thus we revert back to 2500

# Changing the max depth

max_depth: determines how deeply each tree is allowed to grow during any boosting round.


changing the max depth to 2

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 , max_depth = 2) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,  r
                                    andom_state = 42 , max_depth = 2)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Too bad MSE in the above case

Changing the max depth to 200

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 , max_depth = 200) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99, 
                                    random_state = 42 , max_depth = 200)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Changing max_depth = 50

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 , max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99, 
                                    random_state = 42 , max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Changing max_depth = 30

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 , max_depth = 30) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 , max_depth = 30)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Adding Min_child weight =100
default=1

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 ,min_child_weight=100, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 ,min_child_weight=100, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# changing min_child_weight=10

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 ,min_child_weight=10, max_depth = 30) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 ,min_child_weight=10, max_depth = 30)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

changing the  min_child_weight back to 1

# Changing the min_split_loss (gamma)

gamma [default=0, alias: min_split_loss]

gamma: controls whether a given node will split based on the expected reduction in loss after the split. A higher value leads to fewer splits.

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 ,min_child_weight=1,min_split_loss=10, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 ,min_child_weight=1,min_split_loss=10, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

since the MSE is higher than the previous case we change it back to zero (default value)

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,  
                                  random_state = 42 ,min_child_weight=1,min_split_loss=0, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,  
                                    random_state = 42 ,min_child_weight=1,min_split_loss=0, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Effect of Lambda (L2 Regularization) -> changing the value to 0 i.e. NO Regularization
L2 regularization term on weights. Increasing this value will make model more conservative.

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,
                                  reg_lambda=0,  random_state = 42 ,min_child_weight=1,
                                  min_split_loss=0, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,
                                    reg_lambda=0,  random_state = 42 ,min_child_weight=1,
                                    min_split_loss=0, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Thus L2 regularization plays an important role
changing back to the default value of 1

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,
                                  reg_lambda=1,  random_state = 42 ,min_child_weight=1,
                                  min_split_loss=0, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,
                                    reg_lambda=1,  random_state = 42 ,min_child_weight=1,
                                    min_split_loss=0, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Effect of alpha ( L1 regularization)

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,
                                  reg_lambda=0,reg_alpha=1,  random_state = 42 ,min_child_weight=1,
                                  min_split_loss=0, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,
                                    reg_lambda=0,reg_alpha=1,  random_state = 42 ,min_child_weight=1,
                                    min_split_loss=0, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Thus we prefer L2 regularization which is the default value in XGB Python API

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0, learning_rate = 0.99,
                                  reg_lambda=1,reg_alpha=0,  random_state = 42 ,min_child_weight=1,
                                  min_split_loss=0, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0, learning_rate = 0.99,
                                    reg_lambda=1,reg_alpha=0,  random_state = 42 ,min_child_weight=1,
                                    min_split_loss=0, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Tweaking subsample
default=1 and range (0,1)

# Changing the subsample to 0

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0,subsample=0, learning_rate = 0.99,
                                  reg_lambda=1,reg_alpha=0,  random_state = 42 ,min_child_weight=1,
                                  min_split_loss=0, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0,subsample=0, learning_rate = 0.99,
                                    reg_lambda=1,reg_alpha=0,  random_state = 42 ,min_child_weight=1,
                                    min_split_loss=0, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# Changing the subsample to 0.5

In [ ]:
Regressor_Concases = XGBRegressor(n_estimators = 2500, gamma = 0,subsample=0.5, learning_rate = 0.99,
                                  reg_lambda=1,reg_alpha=0,  random_state = 42 ,min_child_weight=1,
                                  min_split_loss=0, max_depth = 50) 
Regressor_fatalities = XGBRegressor(n_estimators = 2500 , gamma = 0,subsample=0.5, 
                                    learning_rate = 0.99,reg_lambda=1,reg_alpha=0,  
                                    random_state = 42 ,min_child_weight=1,min_split_loss=0, max_depth = 50)



import math
from sklearn.metrics import mean_squared_log_error
Regressor_Concases.fit(X,Y1.ravel())
A1 = Regressor_Concases.predict(X)
#B = mean_squared_error(A,Y1)
#print("Training - Mean Squared Error is: ",B) 

A1=abs(A1)
Y1=abs(Y1)


B1 = mean_squared_error(A1,Y1)
B2 = mean_squared_log_error(A1,Y1)
print("-------Confirmed Case---------s")
print("Training - Mean Squared Error is: ",B1)
print("Training - Mean Squared LOG Error is: ",B2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A1,Y1))
display(A1.shape)
#print("Training - ROOT Mean Squared Error is: ",math.sqrt(B1))
#print(rmsle(A1,Y1))


ypred = Regressor_Concases.predict(Xt)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred = round(ypred)
ypred.head(20) 

Regressor_fatalities.fit(X,Y2.ravel())
ypred2= Regressor_fatalities.predict(Xt)
A2 = Regressor_fatalities.predict(X)
A2 = np.round(A2)
A2=abs(A2)
Y2=abs(Y2)

C1 = mean_squared_error(A2,Y2)
C2 = mean_squared_log_error(A2,Y2)
print("--------Fatalities-------")
print("Training - Mean Squared Error is: ",C1)
print("Training - Mean Squared LOG Error is: ",C2)
print("Training - ROOT Mean Squared LOG Error is: ",rmsle(A2,Y2))
display(A2.shape)

ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2 = round(ypred2)
ypred2.head(20) 

# We preserve this subsample rate

# Thus our best performing model settings can be given as follows:
**(n_estimators = 2500, 

**gamma = 0,**

**subsample=0.5, **

**learning_rate = 0.99,
**
reg_lambda=1,

**reg_alpha=0, ** 

**random_state = 42 ,**

**min_child_weight=1,**

**min_split_loss=0, **

**max_depth = 50) ****

In [ ]:
final_pred.to_csv('submission.csv', index=False)

In [ ]:
print("End")